In [1]:
!pip install bertopic[visulaization]

  Using cached bertopic-0.9.4-py2.py3-none-any.whl (57 kB)
  Using cached plotly-5.7.0-py2.py3-none-any.whl (28.8 MB)
  Using cached hdbscan-0.8.28.tar.gz (5.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached sentence_transformers-2.2.0-py3-none-any.whl
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Using cached torch-1.11.0-cp38-cp38-win_amd64.whl (158.0 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-win_amd64.whl (1.1 MB)
  Using cached torchvision-0.12.0-cp38-cp38-win_amd64.whl (1.0 MB)
  Using cached pynndescent-0.5.6-py3-none-any.whl
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp38-cp38-win_amd64.whl size=601943 sha256=657169bf3898b33

In [2]:
!pip install bertopic[visulaization]

In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [48]:
emails = pd.read_csv('C:/Users/Santhosh/Downloads/archive (18)/emails.csv')
#email_subset = emails[:10000]
email_subset = emails.sample(frac=0.02, random_state=1)

print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [49]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [50]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        
    }

In [51]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('');
            
        else:
            results.append(email[key])
    return results

In [53]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head(3))

                                                body
0  It would be nice if you could be at my dinner,...
1  Absolutely.Good point!  Can Peter start to dra...
2  My apologies.  My schedule melted down after w...


In [60]:
docs = list(email_df.loc[:, "body"].values)

In [61]:
docs[:5]

["It would be nice if you could be at my dinner, since I probably won't knowanyone else.  Anytime you want to go to lunch to check on the house status,I'd be glad to go...",
 'Absolutely.Good point!  Can Peter start to draft an override letter?Kay Mann@ENRONDieball/ENRON_DEVELOPMENT@ENRON_DEVELOPMENt, John GRigby/ENRON_DEVELOPMENT@ENRON_DEVELOPMENTAs a reminder to all of us, we will need a form override letter to go withthe form turbine contract.Kay',
 'My apologies.  My schedule melted down after we talked on Monday.  Here\'swhere folks came out.  There\'s some concern about size.  We\'re supposed to beno larger than 3, but we lobbied Aceves and he apparently Ok\'d our"oversized" group.  The other folks in the group--who talked to himoriginally--are pretty sure that five will violate the rules.  Folks wonderedif there were other groups that are smaller than ours that you could hook upwith.  Sorry about that---it\'s a wrinkle that I didn\'t think about when wespoke.  If it gets real ug

In [62]:
model = BERTopic(language="english")

In [63]:
topics, probs = model.fit_transform(docs)

In [64]:
model.get_topic_freq()

,Topic,Count
0,-1,4808
1,0,661
2,1,308
3,2,275
4,3,227
...,...,...
126,125,11
127,126,11
128,127,10
129,128,10


In [65]:
model.get_topic(0)

[('gas', 0.01240090941662693),
 ('deal', 0.010159721384636763),
 ('for', 0.007556868458438333),
 ('price', 0.007317489990527306),
 ('the', 0.007138367260709055),
 ('deals', 0.006810787883810556),
 ('to', 0.00672543160338391),
 ('this', 0.006658241961048837),
 ('on', 0.006624234845862373),
 ('at', 0.00637859975660352)]

In [35]:
model.get_topic(2)

[('you', 0.01583558923809596),
 ('my', 0.012529412411474384),
 ('im', 0.012251673604607464),
 ('so', 0.01108036761056097),
 ('going', 0.010704187044003869),
 ('good', 0.010459053936259132),
 ('it', 0.010224745719187479),
 ('me', 0.010025608876367633),
 ('get', 0.00923041489266652),
 ('just', 0.009081865052696746)]

In [36]:
model.get_topic(5)

[('meeting', 0.022174130028197077),
 ('meetings', 0.012650981335401218),
 ('you', 0.010594609637848211),
 ('will', 0.009604625840187732),
 ('on', 0.009403021296909512),
 ('to', 0.008434077662668592),
 ('attend', 0.008341883123903887),
 ('we', 0.008240044075745847),
 ('week', 0.0076886809042362465),
 ('time', 0.007644085396734864)]

In [2]:
model.visualize_topics()

NameError: name 'model' is not defined